In [1]:
       
## Below code is to detect Face, eye, mouth, nose and detect eye pupil and its movement

import cv2
import dlib
import numpy as np

#start web cam
capture = cv2.VideoCapture(0)

#read the harr_face_detect_classifier.xml
face_cascade = cv2.CascadeClassifier('C:\\Users\\Surendra\\Facial Recognition\\data\\haarcascades\\haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier('C:\\Users\\Surendra\\Facial Recognition\\data\\haarcascades\\haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('C:\\Users\\Surendra\\Facial Recognition\\data\\haarcascades\\haarcascade_smile.xml')
nose_cascade = cv2.CascadeClassifier('C:\\Users\\Surendra\\Facial Recognition\\data\\haarcascades\\haarcascade_mcs_nose.xml')

def print_eye_pos(img, left, right):
    if left == right and left != 0:
        text = ''
        if left == 1:
            print('Looking left')
            text = 'Looking left'
        elif left == 2:
            print('Looking right')
            text = 'Looking right'
        elif left == 3:
            print('Looking up')
            text = 'Looking up'
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(img, text, (30, 30), font,  
                   1, (0, 255, 255), 2, cv2.LINE_AA) 

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    # return the list of (x, y)-coordinates
    return coords

def eye_on_mask(mask, side, shape):
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    mask = cv2.fillConvexPoly(mask, points, 255)
    l = points[0][0]
    t = (points[1][1]+points[2][1])//2
    r = points[3][0]
    b = (points[4][1]+points[5][1])//2
    return mask, [l, t, r, b]

def find_eyeball_position(end_points, cx, cy):
    x_ratio = (end_points[0] - cx)/(cx - end_points[2])
    y_ratio = (cy - end_points[1])/(end_points[3] - cy)
    if x_ratio > 3:
        return 1
    elif x_ratio < 0.33:
        return 2
    elif y_ratio < 0.33:
        return 3
    else:
        return 0

def contouring(thresh, mid, img, end_points, right=False):       
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key = cv2.contourArea)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        if right:
            cx += mid
        cv2.circle(img, (cx, cy), 4, (0, 0, 255), 2)
        pos = find_eyeball_position(end_points, cx, cy)
        return pos
    except:
        pass

def process_thresh(thresh):
    thresh = cv2.erode(thresh, None, iterations=2) 
    thresh = cv2.dilate(thresh, None, iterations=4) 
    thresh = cv2.medianBlur(thresh, 3) 
    thresh = cv2.bitwise_not(thresh)
    return thresh


left = [36, 37, 38, 39, 40, 41]
right = [42, 43, 44, 45, 46, 47]

cv2.namedWindow('image')
kernel = np.ones((9, 9), np.uint8)

def nothing(x):
    pass
cv2.createTrackbar('threshold', 'image', 0, 255, nothing)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:\\Users\\Surendra\\OneDrive\\Desktop\\shape_predictor_68_face_landmarks (1).dat')
    
outer_points = [[49, 59], [50, 58], [51, 57], [52, 56], [53, 55]]
d_outer = [0]*5
inner_points = [[61, 67], [62, 66], [63, 65]]
d_inner = [0]*3
font = cv2.FONT_HERSHEY_SIMPLEX 


while True:
    #read video frame by frame
    ret, frame= capture.read()
    
    if ret is False:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cords = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=1)

    #draw rectange over faces
    for x, y, w, h in face_cords:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255,0), thickness=2)
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        #draw rectange over eyes
        eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.1, minNeighbors=5)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)
        #draw rectange over mouth
        smile = smile_cascade.detectMultiScale(roi_gray,scaleFactor=1.1, minNeighbors=15)
        for (xx, yy, ww, hh) in smile:
            cv2.rectangle(roi_color, (xx, yy), (xx + ww, yy + hh), (0, 255, 0), 2)
        #draw rectange over nose
#        nose = nose_cascade.detectMultiScale(roi_gray, 1.3, 5)
#        for (nx, ny, nw, nh) in nose:
#            cv2.rectangle(roi_color, (nx,ny), (nx+nw,ny+nh), (0, 0, 255), 3)
    
    rects = detector(gray_frame, 1)
    for rect in rects:
        shape = predictor(gray_frame, rect)
        shape = shape_to_np(shape)
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        mask, end_points_left = eye_on_mask(mask, left, shape)
        mask, end_points_right = eye_on_mask(mask, right, shape)
        mask = cv2.dilate(mask, kernel, 5)
        eyes = cv2.bitwise_and(frame, frame, mask=mask)
        mask = (eyes == [0, 0, 0]).all(axis=2)
        eyes[mask] = [255, 255, 255]
        mid = int((shape[42][0] + shape[39][0]) // 2)
        eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
         # increase the threshold in threshold window to detect eye pupi
        threshold = cv2.getTrackbarPos('threshold', 'image')
        _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
        thresh = process_thresh(thresh)
#        print(thresh)
        eyeball_pos_left = contouring(thresh[:, 0:mid], mid, frame, end_points_left)
        print(eyeball_pos_left)
        eyeball_pos_right = contouring(thresh[:, mid:], mid, frame, end_points_right, True)
        print(eyeball_pos_right)
        print_eye_pos(frame, eyeball_pos_left, eyeball_pos_right)
#    cv2.imshow("image", thresh)    
    cnt_outer = 0
    cnt_inner = 0
    for i, (p1, p2) in enumerate(outer_points):
        if d_outer[i] + 3 < shape[p2][1] - shape[p1][1]:
            cnt_outer += 1 
    for i, (p1, p2) in enumerate(inner_points):
        if d_inner[i] + 2 <  shape[p2][1] - shape[p1][1]:
            cnt_inner += 1
    if cnt_outer > 3 and cnt_inner > 2:
        print('Mouth open')
        cv2.putText(frame, 'Mouth open', (30, 30), font,
                1, (0, 255, 255), 2)

    # increase the threshold in threshold window to detect eye pupil
#    cv2.imshow("image", thresh)
        #show face detect Video
    cv2.imshow("Detect face live Video", frame)


#    if cv.waitKey(20) ==ord("e"):              # press e to exit
    if cv2.waitKey(20) > 0 :                    # press any key to exit   
        break

capture.release()
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
Mouth open
None
None
Mouth open
None
None
Mouth open
None
None
Mouth open
None
None
Mouth open
None
None
1
1
Looking left
0
0
0
0
0
0
Mouth open
0
0
Mouth open
None
0
Mouth open
0
3
Mouth open
0
0
0
0
0
0
0
3
Mouth open
3
0
Mouth open
0
0
Mouth open
0
0
0
0
0
0
Mouth open
0
0
0
0
0
0
Mouth open
0
0
0
0
Mouth open
0
0
0
0
Mouth open
0
0
Mouth open
0
0
0
0
0
0
Mouth open
0
0
Mouth open
0
0
Mouth open
0
0
Mouth open
0
0
0
0
0
0
0
0
0
0
0
0
Mouth open
0
0
Mouth open
0
0
0
0
Mouth open
0
0
0
0
0
0
0
0
Mouth open
0
0
0
0
Mouth open
3
3
Looking up
Mouth open
1
1
Looking left
Mouth open
1
1
Looking left
1
1
Looking left
1
1
Looking left
0
1
1
1
Looking left
0
0
0
0
0
0
Mouth open
0
0
0
0
0
0
1
1
Looking left
1
1
Looking left
1
1
Looking left
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [10]:
capture.release()
cv2.destroyAllWindows()